## CS431/631 Data Intensive Distributed Analytics
### Winter 2019 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Hao Dong
* **ID:** 2269784096

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

As you know from the previous assignment, you must tell the Python interpreter where to find Spark before performing any Spark operations in your program.   So, start by doing that:

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

and then create a `SparkContext`:

In [2]:
import random
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [32]:
from collections import defaultdict
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    
    servers = sc.textFile("p2p-Gnutella08-adj.txt")
    num_nodes = servers.count()
    num_edges = servers.map(lambda s : (s.count('	'))).reduce(lambda a,b: a+b)
    return (num_nodes,num_edges)
    
    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    
    servers = sc.textFile("p2p-Gnutella08-adj.txt")
    result = servers.map(lambda s : (s.count('	'),s.split('	')[0]))\
                  .groupByKey().collect()
    result_dic = {}
    for (x,y) in result:
        result_dic[x] = len(y)
    return result_dic
    

def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    
    servers = sc.textFile("p2p-Gnutella08-adj.txt")
    result = servers.flatMap(lambda server : server.split('	')[1:])\
                  .map(lambda node : (node,1))\
                  .reduceByKey(lambda x,y : x+y).collect()
    result_dic = defaultdict(list)
    for (x,y) in result:
        result_dic[y].append(x)
    return result_dic



# num_nodes_edges()
# out_counts()
# in_counts()

---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a positive integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1)

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [254]:
from collections import OrderedDict
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
        
    def server_split(s,source_node_id):
        nodes = [int(x) for x in s.split('	')]
        if nodes[0] == source_node_id : 
            return(nodes[0],[nodes[1:],1])
        return(nodes[0],[nodes[1:],0])
    
    def pprmap(s,source_node_id):
        result = []
        # s:{node_num,[adjacency nodes list,page rank]}
        if len(s[1][0]) == 0:
            result.append((source_node_id,s[1][1]))
        else:
            for x in s[1][0]:
                result.append((x,s[1][1]/len(s[1][0])))
        return result


    servers = sc.textFile("p2p-Gnutella08-adj.txt")
    
    # initialize nodenum, adjacency nodes list and pagerank into a dictionary node_dict
    node_dict = servers.map(lambda s : server_split(s,source_node_id)).collectAsMap()
    
    #do the iterations
    while(num_iterations > 0):
        # calculate the pagerank of each nodes and save it into a dictionary
        pagerank_dict = sc.parallelize(node_dict)\
                          .map(lambda x: (x,node_dict[x]))\
                          .flatMap(lambda s : pprmap(s,source_node_id))\
                          .reduceByKey(lambda x,y : x+y)\
                          .collectAsMap()
        
        # update node_dict
        for nd in node_dict.keys():
            node_dict[nd][1] = jump_factor if (nd == source_node_id) else 0
        for pr in pagerank_dict.keys():
            node_dict[pr][1] += (1 - jump_factor) * pagerank_dict[pr]

        node_dict = OrderedDict(sorted(node_dict.items(), key=lambda e: e[1][1], reverse=True))
        num_iterations -= 1;
        
#         sum = 0
#         for x in node_dict.keys():
#             sum += node_dict[x][1]
        result = []
        i = 0
        for x in node_dict.keys():
            result.append((x,node_dict[x][1]))
            i += 1
            if(i > 9):
                break
    return result

# personalized_page_rank(0,10,0.5) 


---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

If you were unable to get personalized page rank working in Question 2, replace the code cell below with a text (Markdown) cell, and briefly describe how you *would* have modified your program to incorporate a termination condition based on maximum node change, for partial credit. 

In [253]:
from copy import deepcopy
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
  
    def server_split(s,source_node_id):
        nodes = [int(x) for x in s.split('	')]
        if nodes[0] == source_node_id : 
            return(nodes[0],[nodes[1:],1])
        return(nodes[0],[nodes[1:],0])
    
    def pprmap(s,source_node_id):
        result = []
        # s:{node_num,[adjacency nodes list,page rank]}
        if len(s[1][0]) == 0:
            result.append((source_node_id,s[1][1]))
        else:
            for x in s[1][0]:
                result.append((x,s[1][1]/len(s[1][0])))
        return result


    servers = sc.textFile("p2p-Gnutella08-adj.txt")
    
    # initialize nodenum, adjacency nodes list and pagerank into a dictionary node_dict
    node_dict = servers.map(lambda s : server_split(s,source_node_id)).collectAsMap()
    
    #do the iterations
    while(True):
        # calculate the pagerank of each nodes and save it into a dictionary
        pagerank_dict = sc.parallelize(node_dict)\
                          .map(lambda x: (x,node_dict[x]))\
                          .flatMap(lambda s : pprmap(s,source_node_id))\
                          .reduceByKey(lambda x,y : x+y)\
                          .cache()\
                          .collectAsMap()
        
        # update pagerank according to the formula
        new_pagerank_dict = {}
        for nd in node_dict.keys():
            new_pagerank_dict[nd] = jump_factor if (nd == source_node_id) else 0
        for pr in pagerank_dict.keys():
            new_pagerank_dict[pr] += (1 - jump_factor) * pagerank_dict[pr]
        
        # update node_dict and save max change 
        max_change = abs(node_dict[0][1] - new_pagerank_dict[0])
        for nd in node_dict.keys():
            if(abs(node_dict[nd][1] - new_pagerank_dict[nd]) > max_change):
                max_change = abs(node_dict[nd][1] - new_pagerank_dict[nd])
            node_dict[nd][1] = new_pagerank_dict[nd]
        
        # sort node_dict by pagerank
        node_dict = OrderedDict(sorted(node_dict.items(), key=lambda e: e[1][1], reverse=True))

        # print 
        result = []
        i = 0
        for x in node_dict.keys():
            result.append((x,node_dict[x][1]))
            i += 1
            if(i > 9):
                break
        if (max_change < 0.5/servers.count()):
            break
    return result

# import timeit
# start_time = timeit.default_timer()
# personalized_page_rank_stopping_criterion(0,0.5)
# print(timeit.default_timer() - start_time)


---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

I choose to add cache the reduceByKey() RDD. 



---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.